In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras import layers, callbacks
from tensorflow import keras
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
water_quality_raw = pd.read_csv("../input/water-potability/water_potability.csv")
water_quality = water_quality_raw.copy()
water_quality.head()

In [ ]:
good_water_quality = water_quality[water_quality['Potability'] == 1]
good_water_quality = good_water_quality.copy()
bad_water_quality = water_quality[water_quality['Potability'] == 0]
bad_water_quality = good_water_quality.copy()

good_water_quality['ph'] = good_water_quality['ph'].fillna(good_water_quality['ph'].mean())
good_water_quality['Sulfate'] = good_water_quality['Sulfate'].fillna(good_water_quality['Sulfate'].mean())
good_water_quality['Trihalomethanes'] = good_water_quality['Trihalomethanes'].fillna(good_water_quality['Trihalomethanes'].mean())

bad_water_quality['ph'] = bad_water_quality['ph'].fillna(bad_water_quality['ph'].mean())
bad_water_quality['Sulfate'] = bad_water_quality['Sulfate'].fillna(bad_water_quality['Sulfate'].mean())
bad_water_quality['Trihalomethanes'] = bad_water_quality['Trihalomethanes'].fillna(bad_water_quality['Trihalomethanes'].mean())


water_quality = pd.concat([good_water_quality, bad_water_quality])
water_quality = shuffle(water_quality)


In [ ]:
x = water_quality.drop('Potability', axis = 1)
y = water_quality['Potability']

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled, columns=x.columns.values)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.3)
x_train.shape

## Model

In [ ]:
model = keras.Sequential([
    layers.Dense(input_shape = [9], units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 250, activation = 'relu'),
    layers.Dense(units= 1, activation = 'sigmoid')
])

In [ ]:
model.compile(
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
    optimizer = 'adam'
)

In [ ]:
call_back = callbacks.EarlyStopping(
    min_delta=0.001,
    patience= 15,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=20,
        batch_size = 100
)

In [ ]:
history = pd.DataFrame(history.history)
history.head()

In [ ]:
history[['loss', 'val_loss']].plot()

In [ ]:
print('The mean validation accuracy is {}%'.format(history['val_binary_accuracy'].mean() * 100))